In [4]:
import pandas as pd

# Charger le dataset
# Si ton fichier s'appelle exactement 'diabetes.csv'
df = pd.read_csv('diabetes.csv')

# Afficher les informations de base
print("--- Aperçu des données ---")
print(df.head())
print("\n--- Informations sur les colonnes ---")
print(df.info())

--- Aperçu des données ---
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  

--- Informations sur les colonnes ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies       

In [5]:
import numpy as np

# Liste des colonnes où 0 est une erreur médicale
cols_a_corriger = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

# Étape 1 : Remplacer les 0 par "NaN" (Not a Number) pour que Pandas puisse les identifier
for col in cols_a_corriger:
    df[col] = df[col].replace(0, np.nan)

# Étape 2 : Remplacer les NaN par la moyenne de chaque colonne
df.fillna(df.mean(), inplace=True)

# Étape 3 : Vérifier s'il reste des 0 ou des cases vides
print("Nombre de valeurs nulles après correction :")
print(df.isnull().sum())

# Afficher les nouvelles stats pour voir que le 'min' n'est plus à 0
print("\n--- Nouvelles statistiques après nettoyage ---")
print(df[cols_a_corriger].describe())

Nombre de valeurs nulles après correction :
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

--- Nouvelles statistiques après nettoyage ---
          Glucose  BloodPressure  SkinThickness     Insulin         BMI
count  768.000000     768.000000     768.000000  768.000000  768.000000
mean   121.686763      72.405184      29.153420  155.548223   32.457464
std     30.435949      12.096346       8.790942   85.021108    6.875151
min     44.000000      24.000000       7.000000   14.000000   18.200000
25%     99.750000      64.000000      25.000000  121.500000   27.500000
50%    117.000000      72.202592      29.153420  155.548223   32.400000
75%    140.250000      80.000000      32.000000  155.548223   36.600000
max    199.000000     122.000000      99.000000  

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# 1. On sépare les caractéristiques (X) de la réponse (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# 2. On divise : 80% pour apprendre, 20% pour tester
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Création du modèle (on l'appelle 'rf')
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# 4. Entraînement !
rf.fit(X_train, y_train)

# 5. Prédictions sur les données de test
y_pred = rf.predict(X_test)

# 6. Affichage des résultats
print(f"Précision globale : {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\n--- Rapport détaillé ---")
print(classification_report(y_test, y_pred))

Précision globale : 75.32%

--- Rapport détaillé ---
              precision    recall  f1-score   support

           0       0.81      0.81      0.81        99
           1       0.65      0.65      0.65        55

    accuracy                           0.75       154
   macro avg       0.73      0.73      0.73       154
weighted avg       0.75      0.75      0.75       154



In [7]:
# Test sur un nouveau patient imaginaire
# [Grossesses, Glucose, Tension, Peau, Insuline, IMC, Pedigree, Age]
nouveau_patient = [[2, 120, 70, 25, 100, 28.5, 0.5, 35]]

prediction = rf.predict(nouveau_patient)

if prediction[0] == 1:
    print("Résultat : L'IA prédit un risque de Diabète.")
else:
    print("Résultat : L'IA prédit que le patient est en bonne santé.")

Résultat : L'IA prédit que le patient est en bonne santé.


c:\Users\LOUAY\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Phase finale de l'algorithme

In [ ]:
import gradio as gr
import numpy as np

# Fonction que l'interface va appeler
def diagnostiquer(preg, gluc, tens, peau, insu, imc, pedi, age):
    # Organiser les données pour le modèle
    input_data = np.array([[preg, gluc, tens, peau, insu, imc, pedi, age]])
    
    # Faire la prédiction
    prediction = rf.predict(input_data)
    probabilite = rf.predict_proba(input_data)[0][1] * 100 # Optionnel : probabilité
    
    if prediction[0] == 1:
        return f"⚠️ Risque de diabète détecté (Confiance : {probabilite:.1f}%)"
    else:
        return f"✅ Résultat Normal (Confiance : {100 - probabilite:.1f}%)"

# Création de l'interface graphique
demo = gr.Interface(
    fn=diagnostiquer,
    inputs=[
        gr.Number(label="Nombre de grossesses", value=0),
        gr.Slider(40, 200, label="Taux de Glucose", value=120),
        gr.Slider(20, 130, label="Tension Artérielle", value=70),
        gr.Slider(5, 100, label="Épaisseur de la peau (mm)", value=20),
        gr.Slider(0, 800, label="Taux d'Insuline", value=80),
        gr.Number(label="IMC (Indice de Masse Corporelle)", value=25.0),
        gr.Number(label="Indice génétique (Pedigree)", value=0.5),
        gr.Slider(18, 100, label="Âge", value=30)
    ],
    outputs="text",
    title="Assistant Médical - Détection du Diabète",
    description="Entrez les données du patient pour obtenir une prédiction de l'IA basée sur notre modèle Random Forest."
)

# Lancer l'application
demo.launch()

c:\Users\LOUAY\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


c:\Users\LOUAY\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\LOUAY\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\LOUAY\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\LOUAY\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\Users\LOUAY\AppData\Local\Programs\Python\Python313\Lib\site-packages\skl

Created dataset file at: .gradio\flagged\dataset1.csv
